In [2]:
import numpy as np
import hdf5storage
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

mat = hdf5storage.loadmat('Emotrans1_Boy_data_preprocessed_42.mat')
mat2 = hdf5storage.loadmat('Emotrans1_girl_data_preprocessed_42.mat')

input_boy = np.array(mat['All_Feature'][0])
input_girl = np.array(mat2['All_Feature'][0])
total_people = len(input_boy) + len(input_girl)
x_input = np.zeros(shape = (total_people * 192, 2048))
index = 0
trail_num = []   #store the number of trails in different trails type
#store boy data
input_total = np.concatenate([input_boy,input_girl])  

for i in range(len(input_total)):   # number of person    
    for j in range(len(input_total[i])):   # trail type  C D A B
        for k in range(len(input_total[i][j])):   #each trail has k different figure type (neutral or negative) 
            if (j == 1  or j == 0 ):                     
                trail_num.append(len(input_total[i][j][k])) 
                for f in range(len(input_total[i][j][k])): 
                    row = input_total[i][j][k][f]   
                    row = np.array(row)
                    row = row.flatten().transpose()   
                    x_input[index] = row
                    index+=1

#drop unuse row    
x_input = x_input[0:index]

diff_x_input = np.zeros(shape = (int(len(x_input)/4), 2048))
new_index = 0
diff_y_input = np.zeros(shape = (int(len(x_input)/4), 1))
loop_index = 0

for i in range(42):
    index_diff = trail_num[i*8]
    negative_1_mean =(x_input[(loop_index + index_diff*2) : (loop_index + index_diff*3)] + x_input[(loop_index + index_diff*3) : (loop_index + index_diff*4)])/2
    netural_1_mean = (x_input[loop_index : (loop_index + index_diff)] + x_input[(loop_index + index_diff) : (loop_index + index_diff*2)])/2
    diff_x_input[new_index : (new_index + index_diff)] = negative_1_mean - netural_1_mean
    diff_y_input[new_index : (new_index + index_diff)] = 1
    new_index += index_diff
    loop_index += index_diff * 4
    negative_2_mean =(x_input[loop_index : (loop_index + index_diff)] + x_input[(loop_index + index_diff) : (loop_index + index_diff*2)])/2
    netural_2_mean = (x_input[(loop_index + index_diff*2) : (loop_index + index_diff*3)] + x_input[(loop_index + index_diff*3) : (loop_index + index_diff*4)])/2
    diff_x_input[new_index : (new_index + index_diff)] = netural_2_mean - negative_2_mean
    diff_y_input[new_index : (new_index + index_diff)] = 0
    new_index += index_diff
    loop_index += index_diff * 4

diff_x_input = diff_x_input[0:new_index]
diff_y_input = diff_y_input[0:new_index]


FileNotFoundError: [Errno 2] No such file or directory: 'Emotrans1_Boy_data_preprocessed_42.mat'

In [ ]:

lamda = [10,50,500,1000]
for lam in lamda:
    print("the lamda is: ",lam)
    pipe = Pipeline([
                    ('clf', LogisticRegression(solver='saga', 
                                        penalty='l1',
                                        C=lam,
                                        max_iter=1000))
                    ])
    curr_index = 0
    score = []
    for i in range(42):
        index_diff = trail_num[i*8]*2
        test_index = list(range(curr_index,(curr_index + index_diff)))
        X_test = diff_x_input[test_index]
        y_test = diff_y_input[test_index]
        train_index = list(range(len(diff_x_input)))
        train_index[curr_index:(curr_index + index_diff)] = []
        curr_index += index_diff
        X_train = diff_x_input[train_index]
        y_train = diff_y_input[train_index]
        pipe.fit(X_train, y_train.ravel())
        score1 = pipe.score(X_test,y_test.ravel())
        score.append(score1)  
    for i in score:
        print(i)

    print("within subject")
    curr_index = 0
    cross_score = []
    for i in range(42):
        index_diff = trail_num[i*8]*2
        subject_index = list(range(curr_index,(curr_index + index_diff)))
        curr_index += index_diff
        total_x = diff_x_input[subject_index]
        total_y = diff_y_input[subject_index]
        kf = KFold(n_splits=index_diff)
        people_score = []
        for train_index, test_index in kf.split(total_x):
            X_train, X_test = total_x[train_index],total_x[test_index]
            y_train, y_test =total_y[train_index],total_y[test_index]
            pipe.fit(X_train, y_train.ravel())
            score = pipe.score(X_test,y_test.ravel())
            people_score.append(score)
        cross_score.append(sum(people_score)/index_diff)
    for i in cross_score:
        print(i)